# Import libraries 

In [ ]:
import geoip2.database
import ipaddress
import pandas as pd
import logging
from pathlib import Path

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', -1)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

logging.basicConfig(
    level = logging.DEBUG,
    format = '%(asctime)s - %(levelname)s - %(message)s',
    handlers = [logging.StreamHandler()])

# Configuration

In [ ]:
# ../GeoLite2-City.mmdb
city_db_path = Path('') 

# ../GeoLite2-ASN.mmdb
asn_db_path = Path('')

# File with single IP per line
path_to_ips = Path('')

# File to output results to ('Results.xlsx')
path_to_results = Path('')

# Associate IPs with City Information and reformat into Dataframe

In [ ]:
ips = []
internal_ips = []
unparseable_ips = []
line_count = 0

reader = geoip2.database.Reader(city_db_path)
with open(path_to_ips) as f:
    for line in f:
        line_count += 1
        try:
            if not ipaddress.ip_address(line.strip()).is_private:
                ips.append(reader.city(line.strip()))
            else:
                internal_ips.append(line.strip())
        except Exception as e:
            logging.info(f"Could not process {line.strip()}: {e}")
            unparseable_ips.append(line.strip())

In [ ]:
print(f"Parsed a total of {line_count - len(internal_ips)} from {line_count} provided ips.")

if internal_ips:
    print(f"Printing {len(internal_ips)} private ips left unparsed:")
    for ip in internal_ips:
        print(ip)
    
if unparseable_ips:
    print(f"Printing {len(unparseable_ips)} unparseable ips:")
    for ip in unparseable_ips:
        print(ip)

In [ ]:
parsed_ips = []
for ip in ips:
    parsed_ips.append(f"{ip.traits.ip_address}\t{ip.country.name}\t{ip.city.name}")
    
df = pd.DataFrame([sub.split("\t") for sub in parsed_ips], columns=['ip_address', 'country', 'city'])

# Associate IPs with ASN Information, reformat into Dataframe

In [ ]:
asns = []
internal_ips = []
unparseable_ips = []
line_count = 0

reader2 = geoip2.database.Reader(asn_db_path)
with open(path_to_ips) as f:
    for line in f:
        line_count += 1
        try:
            if not ipaddress.ip_address(line.strip()).is_private:
                asns.append(reader2.asn(line.strip()))
            else:
                internal_ips.append(line.strip())
        except Exception as e:
            logging.info(f"Could not process {line.strip()}: {e}")
            unparseable_ips.append(line.strip())

In [ ]:
print(f"Parsed a total of {line_count - len(internal_ips)} from {line_count} provided ips.")

if internal_ips:
    print(f"Printing {len(internal_ips)} private ips left unparsed:")
    for ip in internal_ips:
        print(ip)
    
if unparseable_ips:
    print(f"Printing {len(unparseable_ips)} unparseable ips:")
    for ip in unparseable_ips:
        print(ip)

In [ ]:
parsed_asns = []
for ip in asns:
    parsed_asns.append(f"{ip.ip_address}\t{ip.autonomous_system_number}\t{ip.autonomous_system_organization}")
    
df2 = pd.DataFrame([sub.split("\t") for sub in parsed_asns], columns=['ip_address', 'asn_system_number', 'asn_org'])

# Merge Dataframes

In [ ]:
df3 = pd.merge(df, df2, on='ip_address', how='outer')

# Analysis 

In [ ]:
countryAsnGroupby = df3.groupby(['country', 'asn_org'])['ip_address'].count().reset_index().sort_values(by='ip_address', ascending=False)

In [ ]:
countryGroupby = df3.groupby(['country'])['ip_address'].count().reset_index().sort_values(by='ip_address', ascending=False)

# Write Results to Excel 

In [ ]:
writer = pd.ExcelWriter(path_to_results, engine='openpyxl')
df3.to_excel(writer, sheet_name='GeolocatedIPs', index=False)
countryAsnGroupby.to_excel(writer, sheet_name='CountryASNGroupby', index=False)
countryGroupby.to_excel(writer, sheet_name='CountryGroupby', index=False)
writer.save()